In [19]:
import json
import re
import os


In [20]:
# --- Bloque para definir rutas dinámicamente ---

# El notebook está en: .../QLAB_CHATBOT_CORRUPCION/procesamiento/src_json_optimizado/
notebook_dir = os.getcwd()

# Subimos dos niveles para llegar a la raíz del proyecto: QLAB_CHATBOT_CORRUPCION/
# 1. de src_json_optimizado a procesamiento ('..')
# 2. de procesamiento a QLAB_CHATBOT_CORRUPCION ('..')
project_root = os.path.abspath(os.path.join(notebook_dir, '..', '..'))

# Nombres de los archivos
consolidated_filename = "salida_informes_consolidados_desde_csv.jsonl" # Tu archivo de entrada
chunked_filename = "salida_chunks_final.jsonl" # Tu archivo de salida

# Construir rutas completas a los archivos en la carpeta 'output' del proyecto
consolidated_file_path = os.path.join(project_root, 'output', consolidated_filename)
chunked_output_file_path = os.path.join(project_root, 'output', chunked_filename)

# (Opcional) Imprime las rutas para verificar:
print(f"Ruta del archivo consolidado de entrada: {consolidated_file_path}")
print(f"Ruta del archivo de chunks de salida: {chunked_output_file_path}")
# --- Fin del bloque de rutas ---

Ruta del archivo consolidado de entrada: c:\Users\LENOVO\Documents\GitHub\qlab_chatbot_corrupcion\output\salida_informes_consolidados_desde_csv.jsonl
Ruta del archivo de chunks de salida: c:\Users\LENOVO\Documents\GitHub\qlab_chatbot_corrupcion\output\salida_chunks_final.jsonl


In [21]:

def parse_text_block_into_items(text_block, item_prefix_regex_str=None, default_if_no_prefix=True):
    """
    Divide un bloque de texto en ítems.
    Si item_prefix_regex_str es None o no encuentra matches, y default_if_no_prefix es True,
    trata todo el bloque como un solo ítem.
    """
    items = []
    if not text_block or pd.isna(text_block): # Manejar None o NaN explícitamente
        return items
        
    text_block = str(text_block).strip()
    if not text_block: # Si después de strip es vacío
        return items

    if item_prefix_regex_str:
        # Encontrar todos los inicios de ítems y sus textos
        matches = list(re.finditer(item_prefix_regex_str, text_block, re.MULTILINE))
        
        if matches:
            for i, match_obj in enumerate(matches):
                # El texto del ítem es desde el final del prefijo actual
                # hasta el inicio del prefijo del siguiente ítem (o el final del bloque)
                text_start_index = match_obj.end() # Comienza después del prefijo
                
                text_end_index = matches[i+1].start() if i + 1 < len(matches) else len(text_block)
                
                item_text = text_block[text_start_index:text_end_index].replace("\n", " ").strip()
                if item_text: # Solo añadir si no está vacío
                    items.append(item_text)
            return items

    # Si no hay prefijo o no se encontraron matches con el prefijo, y default_if_no_prefix es True
    if default_if_no_prefix and text_block:
        return [text_block.replace("\n", " ").strip()]
    
    return items


def chunk_consolidated_reports(consolidated_jsonl_path, chunked_jsonl_path):
    """
    Lee un JSONL de informes consolidados y genera un nuevo JSONL con chunks individuales
    para objetivos, observaciones y recomendaciones.
    """
    
    # Patrones regex para identificar ítems dentro de los bloques de texto
    # (ajustar según la estructura real de tus bloques de texto)
    # Para objetivos específicos que empiezan con guion:
    obj_especifico_pattern = r"^\s*-\s+"
    # Para observaciones/recomendaciones numeradas:
    obs_rec_pattern = r"^\s*\d+\s*[\.-]\s*"


    with open(consolidated_jsonl_path, 'r', encoding='utf-8') as infile, \
         open(chunked_jsonl_path, 'w', encoding='utf-8') as outfile:

        for line in infile:
            try:
                report_data = json.loads(line)
            except json.JSONDecodeError as e:
                print(f"Error decodificando JSON en la línea: {line.strip()}. Error: {e}")
                continue

            report_id = report_data.get("report_id", "ID_DESCONOCIDO")
            base_metadata = {
                "numero_informe": report_data.get("numero_informe"),
                "titulo_informe": report_data.get("titulo_informe"),
                "region": report_data.get("region"),
                "provincia": report_data.get("provincia"),
                "distrito": report_data.get("distrito"),
                "entidad_auditada": report_data.get("entidad_auditada"),
                "year": report_data.get("year"),
                "periodo_inicio": report_data.get("periodo_inicio"),
                "periodo_fin": report_data.get("periodo_fin"),
                "monto_auditado": report_data.get("monto_auditado"),
                "monto_examinado": report_data.get("monto_examinado"),
                "modalidad": report_data.get("modalidad"),
                "fecha_emision": report_data.get("fecha_emision"),
                "unidad_emite": report_data.get("unidad_emite"),
                "personas_implicadas": report_data.get("personas_implicadas_consolidado", []),
                # Incluir responsabilidades consolidadas
                **report_data.get("responsabilidades_consolidadas", {}) # Desempaqueta el dict de responsabilidades
            }
            
            chunk_idx_counter = {"obj": 0, "obs": 0, "rec": 0}

            # 1. Procesar Objetivos
            objetivos_block = report_data.get("texto_objetivos_completo", "")
            if objetivos_block:
                # Intentar separar objetivo general de específicos
                # Esta es una lógica simple, puede necesitar ajustes robustos
                general_obj_text = ""
                especificos_block_text = objetivos_block # Por defecto, todo es específico si no se encuentra "general"

                # Patrones para encontrar "Objetivo General" y "Objetivos Específicos"
                # Se hacen no sensibles a mayúsculas/minúsculas con re.IGNORECASE
                match_general = re.search(r"(Objetivo\s+General[:\s\n]+|2\.1\s*Objetivo\s*general\s*\n)([\s\S]+?)(?=(Objetivos\s+Específicos[:\s\n]+|2\.2\s*Objetivos\s*específicos|\Z))", objetivos_block, re.IGNORECASE | re.DOTALL)
                
                if match_general:
                    general_obj_text = match_general.group(2).replace("\n", " ").strip()
                    # El resto sería para específicos, si "Objetivos Específicos" sigue
                    idx_fin_general = match_general.end()
                    match_especificos_header = re.search(r"(Objetivos\s+Específicos[:\s\n]+|2\.2\s*Objetivos\s*específicos\s*\n)", objetivos_block[idx_fin_general:], re.IGNORECASE)
                    if match_especificos_header:
                         especificos_block_text = objetivos_block[idx_fin_general + match_especificos_header.end():]
                    else: # No hay encabezado de específicos, o el general fue el último
                         especificos_block_text = "" 
                
                if general_obj_text:
                    chunk_id = f"{report_id}_obj_{chunk_idx_counter['obj']}"
                    chunk_data = {
                        "chunk_id": chunk_id, "report_id": report_id,
                        "chunk_text": general_obj_text, "source_field": "objetivo",
                        "metadata": base_metadata.copy()
                    }
                    outfile.write(json.dumps(chunk_data, ensure_ascii=False) + '\n')
                    chunk_idx_counter['obj'] += 1
                
                # Procesar objetivos específicos
                if especificos_block_text:
                    lista_obj_especificos = parse_text_block_into_items(especificos_block_text, obj_especifico_pattern)
                    for obj_esp_text in lista_obj_especificos:
                        if not obj_esp_text: continue
                        chunk_id = f"{report_id}_obj_{chunk_idx_counter['obj']}"
                        chunk_data = {
                            "chunk_id": chunk_id, "report_id": report_id,
                            "chunk_text": obj_esp_text, "source_field": "objetivo",
                            "metadata": base_metadata.copy()
                        }
                        outfile.write(json.dumps(chunk_data, ensure_ascii=False) + '\n')
                        chunk_idx_counter['obj'] += 1
                elif not general_obj_text and objetivos_block: # Si no se separó general y hay texto, tratarlo como un solo objetivo
                    chunk_id = f"{report_id}_obj_{chunk_idx_counter['obj']}"
                    chunk_data = {
                        "chunk_id": chunk_id, "report_id": report_id,
                        "chunk_text": objetivos_block.replace("\n", " ").strip(), "source_field": "objetivo",
                        "metadata": base_metadata.copy()
                    }
                    outfile.write(json.dumps(chunk_data, ensure_ascii=False) + '\n')
                    chunk_idx_counter['obj'] += 1


            # 2. Procesar Observaciones
            observaciones_block = report_data.get("texto_observaciones_completo", "")
            if observaciones_block:
                lista_observaciones = parse_text_block_into_items(observaciones_block, obs_rec_pattern)
                for obs_text in lista_observaciones:
                    if not obs_text: continue
                    chunk_id = f"{report_id}_obs_{chunk_idx_counter['obs']}"
                    chunk_data = {
                        "chunk_id": chunk_id, "report_id": report_id,
                        "chunk_text": obs_text, "source_field": "observacion",
                        "metadata": base_metadata.copy()
                    }
                    outfile.write(json.dumps(chunk_data, ensure_ascii=False) + '\n')
                    chunk_idx_counter['obs'] += 1

            # 3. Procesar Recomendaciones
            recomendaciones_block = report_data.get("texto_recomendaciones_completo", "")
            if recomendaciones_block:
                # La recomendación puede tener un destinatario antes de la numeración.
                # Intentaremos quitarlo si existe "Al [Cargo/Entidad]:" o similar antes de pasar a parse_text_block_into_items
                # O ajustar parse_text_block_into_items para manejarlo.
                # Por ahora, se asume que el patrón numérico es lo principal.
                lista_recomendaciones = parse_text_block_into_items(recomendaciones_block, obs_rec_pattern)
                for rec_text in lista_recomendaciones:
                    if not rec_text: continue
                    # Quitar posible destinatario al inicio si el patrón es "Al ... N. texto"
                    # Esto es opcional y depende de cómo quieras el chunk_text final
                    # rec_text_cleaned = re.sub(r"^(Al\s+.*?[\n:])?\s*\d+\s*[\.-]\s*", "", rec_text, flags=re.IGNORECASE).strip()
                    # if not rec_text_cleaned: rec_text_cleaned = rec_text # Si la limpieza lo dejó vacío, usar el original

                    chunk_id = f"{report_id}_rec_{chunk_idx_counter['rec']}"
                    chunk_data = {
                        "chunk_id": chunk_id, "report_id": report_id,
                        "chunk_text": rec_text, # Usar rec_text o rec_text_cleaned
                        "source_field": "recomendacion",
                        "metadata": base_metadata.copy()
                    }
                    outfile.write(json.dumps(chunk_data, ensure_ascii=False) + '\n')
                    chunk_idx_counter['rec'] += 1
                    
    print(f"Archivo JSONL con chunks individuales '{chunked_jsonl_path}' generado exitosamente.")



In [22]:
# Llamada a la función principal
chunk_consolidated_reports(consolidated_file_path, chunked_output_file_path)

Archivo JSONL con chunks individuales 'c:\Users\LENOVO\Documents\GitHub\qlab_chatbot_corrupcion\output\salida_chunks_final.jsonl' generado exitosamente.
